<a href="https://colab.research.google.com/github/nitchie/notebook_NK225op/blob/master/225op.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

import os
import re
import requests
from bs4 import BeautifulSoup
from google.colab import drive

def debug_print(str):
  print(str)
  #pass

#https://stackoverflow.com/questions/46986398/import-data-into-google-colaboratory
drive.mount('/content/drive')

storage_dir =  'drive/My Drive/225op/'

page_urls = [
    'https://www.jpx.co.jp/markets/derivatives/perticipant-volume/index.html'
    ,'https://www.jpx.co.jp/markets/derivatives/perticipant-volume/archives-01.html'
    ,'https://www.jpx.co.jp/markets/derivatives/perticipant-volume/archives-02.html'
    ,'https://www.jpx.co.jp/markets/derivatives/perticipant-volume/archives-03.html'
#    ,'https://www.jpx.co.jp/markets/derivatives/perticipant-volume/archives-11.html'
]
csvfile_list = []

for page_url in page_urls:
  soup = BeautifulSoup(requests.get(page_url).content,'html.parser')
  elements = soup.find_all(href=re.compile("csv"))
  if not elements:
    debug_print('page '+page_url+' is not found.')
  for el in elements:
    csv_url = el['href']
    #/markets/derivatives/perticipant-volume/nlsgeu0000034wa2-att/20180531_volume_by_participant_night.csv
    #/markets/derivatives/perticipant-volume/nlsgeu0000034wa2-att/20180531_volume_by_participant_night_J-NET.csv
    #/markets/derivatives/perticipant-volume/nlsgeu0000034wa2-att/20180531_volume_by_participant_whole_day.csv
    #/markets/derivatives/perticipant-volume/nlsgeu0000034wa2-att/20180531_volume_by_participant_whole_day_J-NET.csv
    m = re.match(r'/markets/derivatives/perticipant-volume/.+/(.+)',csv_url)
    if m:
      csvfile = m[1]
      csvfile_list = csvfile_list + [csvfile]
      outfile = storage_dir+csvfile
      
      if not os.path.exists(outfile):
          with open(outfile,'w') as f:
            if f:
                response = requests.get('https://www.jpx.co.jp'+csv_url)
                f.write(response.text)
                debug_print('save '+outfile)
    #break
    


In [0]:
  import csv 
  
  jpxcodes = {}
  companys = {}
  positions = {}
  
  for csvfile in csvfile_list:
    with open(storage_dir+csvfile) as f:
      if f:
        lines = list(csv.reader(f))
        jpxcode = ''
        instrument = ''
        tradedate = lines[1][3]
        if re.search(r'\d{8}',tradedate):
          # debug_print(csvfile+" "+tradedate)
          for line in lines:
            if line:
              if line[0] == 'JPX Code':
                jpxcode = line[1]
              elif line[0] == 'Instrument':
                instrument = line[1]
                jpxcodes[jpxcode] = instrument
              else:
                if re.search(r'^\d+$',line[0]):
                  company_code = line[0]
                  company_name_j = line[1]
                  company_name_e = line[2]
                  volume = int(line[3])
                  companys[company_code] = company_name_e
                  if not instrument in positions:
                    positions[instrument] = {}
                  if not company_name_e in positions[instrument]:
                    positions[instrument][company_name_e] = {'net':0,'long': 0,'short':0}
                  positions[instrument][company_name_e]['long'] += volume
                  positions[instrument][company_name_e]['net'] = positions[instrument][company_name_e]['long'] - positions[instrument][company_name_e]['short']
                if len(line) > 4 and re.search(r'^\d+$',line[4]):
                  company_code = line[4]
                  company_name_j = line[5]
                  company_name_e = line[6]
                  volume = int(line[7])
                  if not company_name_e in positions[instrument]:
                    positions[instrument][company_name_e] = {'net':0,'long': 0,'short':0}
                  positions[instrument][company_name_e]['short'] += volume
                  positions[instrument][company_name_e]['net'] = positions[instrument][company_name_e]['long'] - positions[instrument][company_name_e]['short']
        else:
          pass # show error


In [0]:
positions

In [0]:
from pandas import DataFrame,Series
DataFrame(positions)
